In [ ]:
import serial
import matplotlib.pyplot as plt
import numpy as np 
import time
import math
import csv
import openai
import os

def append_new_data_to_csv(filename, resistance_names, new_data):
    
    file_exists = os.path.exists(filename)
    mode = 'a' if file_exists else 'w'

    with open(filename, mode, newline='') as file:
        writer = csv.writer(file)

        if not file_exists:
           
            writer.writerow(['Resistance Name'] + resistance_names)

        
        writer.writerow(['New Measurement'] + new_data)



port0 = 'COM3'  
port1 = 'COM4'
baudrate = 9600  


try:
    ser0 = serial.Serial(port0, baudrate)
    ser1 = serial.Serial(port1, baudrate)
    ser0.close()
    ser1.close()
except serial.SerialException:
    print("请关闭串口连接后再运行程序！")
    exit()


ser0 = serial.Serial(port0, baudrate)
ser1 = serial.Serial(port1, baudrate)


start_time = time.time()

# python 
time_data = []


data_collection_time = 40 


while (time.time() - start_time) < data_collection_time:
# while True:
    if ser0.in_waiting > 0:        
        
            data0 =ser0.readline().decode('utf-8').rstrip()
            data1 =ser1.readline().decode('utf-8').rstrip()
            
            try:    

                
                resistance_names = [
                    "R_B21C12", "R_A11C12", "R_A11B21", "R_B22C12", "R_A21B22", "R_A21C12", 
                    "R_A21C11", "R_A21B12", "R_B12C11", "R_A22C22", "R_A22B22", "R_B22C22",
                    "R_A22B12", "R_A22C21", "R_B12C21", "R_A12C21", "R_A12B11", "R_B11C21",
                    "R_A12B21", "R_A12C22", "R_B21C22", "R_A11B11", "R_A11C11", "R_B11C11"
                ]

                
                resistances = data0.split(',') + data1.split(',')

                
                resistances = [float(r) for r in resistances]

                
                filename = "resistances.csv"

                
                append_new_data_to_csv(filename, resistance_names, resistances)

                print("New data appended to CSV successfully.")

            except ValueError:
                print("Invalid sensor value.")
            except Exception as e:
                print(f"An error occurred: {e}")       

ser0.close()
ser1.close()

In [ ]:
import numpy as np

def load_data_from_csv(filename):
    try:
        
        data = np.genfromtxt(filename, delimiter=',', skip_header=1, usecols=range(1, 25))
        return data
    except Exception as e:
        print(f"An error occurred while loading the data: {e}")
        return None


filename = "resistances.csv"


resistance_data = load_data_from_csv(filename)

if resistance_data is not None:
    print("Data loaded into NumPy array successfully.")
    print(resistance_data)
else:
    print("Failed to load data.")


In [ ]:
import serial
import numpy as np
import openai


openai.api_key = 'your-fine-tuned-model-api-key'


fine_tuned_model_name = "havronl"

def generate_explanation(data_analysis):
    messages = [
        {"role": "system", "content": "You are an expert data analyst."},
        {"role": "user", "content": f"Given the following sensor data analysis results: {data_analysis}, explain what these results might indicate about the sensor's performance or environmental conditions."}
    ]

    response = openai.ChatCompletion.create(
        model=fine_tuned_model_name,
        messages=messages,
        max_tokens=150
    )
    return response.choices[0].message['content']

def generate_prediction(data_trends):
    messages = [
        {"role": "system", "content": "You are an AI trained to provide predictions and suggestions based on sensor data trends."},
        {"role": "user", "content": f"Based on these data trends from a sensor system: {data_trends}, predict potential future issues that might arise and suggest preventive measures."}
    ]

    response = openai.ChatCompletion.create(
        model=fine_tuned_model_name,
        messages=messages,
        max_tokens=200
    )
    return response.choices[0].message['content']


sensor_data = resistance_data

for i in range(resistance_data.shape[1]):
    sensor_data = resistance_data[:, i]
    data_analysis = {
        "mean": np.mean(sensor_data),
        "std_deviation": np.std(sensor_data),
        "max_value": np.max(sensor_data),
        "min_value": np.min(sensor_data)
    }
    
    print(f"Sensor {i+1} Data Analysis:", data_analysis)

   
    explanation = generate_explanation(data_analysis)
    print(f"Sensor {i+1} Model Explanation:", explanation)

   
    data_trends = f"The average resistance for Sensor {i+1} was {data_analysis['mean']:.2f} MΩ, with a standard deviation of {data_analysis['std_deviation']:.2f} MΩ. The maximum and minimum values recorded were {data_analysis['max_value']:.2f} MΩ and {data_analysis['min_value']:.2f} MΩ, respectively."

    
    prediction_and_suggestions = generate_prediction(data_trends)
    print(f"Sensor {i+1} Predictions and Suggestions:", prediction_and_suggestions)
